# TWPA Wrapper Basic Tests for JosephsonCircuits.jl

This notebook handles the Julia simulation of nonlinear circuits using Kevin O'Brien's JosephsonCircuits.jl package.  
It contains basic tests showing that the updated version of JosephsonCircuits.jl is running correctly.  

In [1]:
# Cell 1a: Initial Setup (Run ONCE per session)
# Julia configuration is controlled by julia_setup.py constants:
# - USE_LOCAL_FORK: Toggle local development vs remote
# - USE_GITHUB_FORK: Toggle GitHub fork vs registered (when USE_LOCAL_FORK=False)

from twpa_design import julia_setup

# Initialize julia_ready flag
julia_ready = False

# Setup Julia (configuration is in julia_setup.py)
print("🚀 Setting up Julia...")
jl = julia_setup.get_julia_for_session()

# Try to load JosephsonCircuits
try:
    jl.eval("using JosephsonCircuits")
    
    # Verify what version is loaded
    actual_path = jl.eval("pathof(JosephsonCircuits)")
    print(f"✅ SUCCESS! Using JosephsonCircuits from: {actual_path}")
    
    # Use helper function to check which version (no hardcoded UUIDs!)
    source_type = julia_setup.check_package_source(jl)
    
    if source_type == "local":
        print("🔥 Hot-reloading enabled - edit .jl files and changes auto-reload!")
    elif source_type == "github":
        print("📦 Using GitHub fork with Taylor expansion feature")
    elif source_type == "registered":
        print("📦 Using registered version")
    else:
        print("⚠️ Could not determine package source")
    
    julia_ready = True
        
except Exception as e:
    print(f"❌ Failed to load JosephsonCircuits: {e}")
    print("Check julia_setup.py configuration")
    julia_ready = False

🚀 Setting up Julia...
🚀 Initializing new Julia session...
📍 Using Julia from: C:\Users\mbm2\.julia\juliaup\julia-1.10.0+0.x64.w64.mingw32
Julia version: julia version 1.10.0
⬆️  Upgraded to Julia 1.11.5
📦 Using GitHub fork: https://github.com/MaxMalnou/JosephsonCircuits.jl
   Branch: taylor-expansion-feature
  🔍 Checking if GitHub fork is already installed...
  ✅ GitHub fork already installed, skipping clone
  📦 Loading JosephsonCircuits...
  🔍 Verifying installation...
  📍 Package loaded from: C:\Users\mbm2\.julia\packages\JosephsonCircuits\bUSDh\src\JosephsonCircuits.jl
  ✅ Successfully using GitHub fork!
✅ SUCCESS! Using JosephsonCircuits from: C:\Users\mbm2\.julia\packages\JosephsonCircuits\bUSDh\src\JosephsonCircuits.jl
📦 Using GitHub fork with Taylor expansion feature


In [ ]:
# Cell 1b: Switch Julia Version (Optional - requires kernel restart first)
# 
# To switch between local fork / GitHub fork / registered version:
# 1. Restart the kernel
# 2. Edit julia_setup.py and change the configuration constants:
#    - USE_LOCAL_FORK = True/False
#    - USE_GITHUB_FORK = True/False
# 3. Re-run Cell 1a
#
# Configuration options:
# - USE_LOCAL_FORK=True  → Local development with hot-reloading
# - USE_LOCAL_FORK=False, USE_GITHUB_FORK=True  → GitHub PR branch
# - USE_LOCAL_FORK=False, USE_GITHUB_FORK=False → Kevin's registered version

print("ℹ️ To switch Julia version:")
print("1. Restart kernel")
print("2. Edit USE_LOCAL_FORK and USE_GITHUB_FORK in julia_setup.py")
print("3. Re-run Cell 1a")

In [2]:
julia_setup.force_revise_update(jl)
result = jl.eval("""
    using JosephsonCircuits
    
    # Same circuits as before
    circuit_jj = [
        ("P1", "1", "0", 1),
        ("R1", "1", "0", 50.0),
        ("Lj1", "1", "2", 329e-12),
        ("C1", "2", "0", 132e-15),
        ("R2", "2", "0", 50.0),
    ]
    
    circuit_taylor = [
        ("P1", "1", "0", 1),
        ("R1", "1", "0", 50.0),
        ("NL1", "1", "2", "poly 329e-12, 0.0, 0.5"),
        ("C1", "2", "0", 132e-15),
        ("R2", "2", "0", 50.0),        
    ]
    
    circuitdefs = Dict()
    wp = 2*pi*10e9
    currents = [0.001e-6, 0.05e-6, 0.1e-6]  # Different pump powers
    
    results = []
    for Ip in currents
        # Solve with 4 harmonics to see 1st and 3rd
        result_jj = hbnlsolve((wp,), (4,), [(mode=(1,), port=1, current=Ip)], 
                             circuit_jj, circuitdefs; dc=false, odd=true, even=false,
                             keyedarrays=Val(false))
        
        result_taylor = hbnlsolve((wp,), (4,), [(mode=(1,), port=1, current=Ip)], 
                                 circuit_taylor, circuitdefs; dc=false, odd=true, even=false,
                                 keyedarrays=Val(false))
        
        # Get branch fluxes
        branch_flux_jj = result_jj.Rbnm * result_jj.nodeflux
        branch_flux_taylor = result_taylor.Rbnm * result_taylor.nodeflux
        
        # We know from diagnostic that nonlinear element is on branch 2
        nl_branch = 2
        
        # With odd=true and 4 harmonics, we get modes (1,) and (3,)
        # Each branch has Nmodes entries
        Nmodes_jj = length(result_jj.frequencies.modes)
        Nmodes_taylor = length(result_taylor.frequencies.modes)
        
        # For branch 2: indices are (branch-1)*Nmodes + mode_index
        # Mode 1 (fundamental): index 1
        # Mode 3 (3rd harmonic): index 2
        flux_jj_fund = branch_flux_jj[(nl_branch-1)*Nmodes_jj + 1]
        flux_taylor_fund = branch_flux_taylor[(nl_branch-1)*Nmodes_taylor + 1]
        flux_jj_3h = branch_flux_jj[(nl_branch-1)*Nmodes_jj + 2]
        flux_taylor_3h = branch_flux_taylor[(nl_branch-1)*Nmodes_taylor + 2]
        
        push!(results, Dict(
            "Ip_nA" => Ip * 1e9,
            "flux_jj_fund" => flux_jj_fund,
            "flux_taylor_fund" => flux_taylor_fund,
            "flux_jj_fund_mag" => abs(flux_jj_fund),
            "flux_taylor_fund_mag" => abs(flux_taylor_fund),
            "fund_diff_percent" => 100 * abs(abs(flux_jj_fund) - abs(flux_taylor_fund)) / abs(flux_jj_fund),
            "flux_jj_3h" => flux_jj_3h,
            "flux_taylor_3h" => flux_taylor_3h,
            "flux_jj_3h_mag" => abs(flux_jj_3h),
            "flux_taylor_3h_mag" => abs(flux_taylor_3h),
            "harmonic_ratio_jj" => abs(flux_jj_3h) / abs(flux_jj_fund),
            "harmonic_ratio_taylor" => abs(flux_taylor_3h) / abs(flux_taylor_fund)
        ))
    end
    
    results
""")

print("="*60)
print("CORRECTED TAYLOR APPROXIMATION TEST")
print("="*60)
print("Testing L = L0*(1 + 0.5*φ²)")
print("-"*80)
print(f"{'Ip (nA)':<10} {'|φ| JJ':<12} {'|φ| Taylor':<12} {'Diff %':<10} {'|φ3| JJ':<12} {'|φ3| Taylor':<12}")
print("-"*80)

for r in result:
    print(f"{r['Ip_nA']:<10.1f} {r['flux_jj_fund_mag']:<12.6f} {r['flux_taylor_fund_mag']:<12.6f} "
          f"{r['fund_diff_percent']:<10.2f} {r['flux_jj_3h_mag']:<12.6e} {r['flux_taylor_3h_mag']:<12.6e}")

print(f"\nFundamental flux values (complex):")
for r in result:
    print(f"  {r['Ip_nA']:.1f} nA: JJ = {r['flux_jj_fund']}, Taylor = {r['flux_taylor_fund']}")

# Also check what's happening with the Taylor nonlinearity
result2 = jl.eval("""
    # Let's trace through what calc_taylor_contributions! does
    # For I = c1*φ + c3*φ³, with our coefficients:
    phi0 = JosephsonCircuits.phi0
    L0 = 329e-12
    c1 = phi0 / L0  # Linear coefficient
    c3 = -phi0 * 0.5 / L0  # Cubic coefficient from our c2
    
    # For comparison, Josephson: I = (φ0/Lj)*sin(φ)
    # Taylor expansion: sin(φ) ≈ φ - φ³/6
    # So we want: c3 = -(φ0/L0)/2
    ideal_c3 = -(phi0/L0) / 2
    
    Dict(
        "c1" => c1,
        "c3" => c3,
        "ideal_c3" => ideal_c3,
        "c3_ratio" => c3 / ideal_c3
    )
""")

print(f"\n{'='*60}")
print("COEFFICIENT CHECK")
print(f"{'='*60}")
print(f"Linear coefficient c1: {result2['c1']:.6e}")
print(f"Cubic coefficient c3: {result2['c3']:.6e}")
print(f"Ideal c3 for sin(φ): {result2['ideal_c3']:.6e}")
print(f"Ratio c3/ideal: {result2['c3_ratio']:.3f}")


print("\n" + "="*50)
print("FINAL STATUS")
print("="*50)

⚠️ Revise.jl not loaded. Loading it now...
✅ Forced Revise update complete - changes should be reflected now
CORRECTED TAYLOR APPROXIMATION TEST
Testing L = L0*(1 + 0.5*φ²)
--------------------------------------------------------------------------------
Ip (nA)    |φ| JJ       |φ| Taylor   Diff %     |φ3| JJ      |φ3| Taylor 
--------------------------------------------------------------------------------
1.0        0.000539     0.000539     0.00       0.000000e+00 0.000000e+00
50.0       0.026967     0.026967     0.00       3.048187e-06 3.048849e-06
100.0      0.053992     0.053992     0.00       2.449804e-05 2.451945e-05

Fundamental flux values (complex):
  1.0 nA: JJ = (-0.0005388571385098259+1.7327218024950014e-05j), Taylor = (-0.0005388571385098259+1.7327218024950014e-05j)
  50.0 nA: JJ = (-0.026952511798355415+0.0008688590528924123j), Taylor = (-0.02695251296911725+0.0008688593564098179j)
  100.0 nA: JJ = (-0.053963214065672616+0.0017528089747941428j), Taylor = (-0.0539632519264

In [3]:
julia_setup.force_revise_update(jl)
jl.eval("JosephsonCircuits.clear_debug_log()")
result = jl.eval(r"""
    using JosephsonCircuits
    
    # Circuit 1: Single LC stage with JJ
    circuit_l = [
        ("P1", "1", "0", 1),
        ("R1", "1", "0", 50.0),
        ("Lj1", "1", "2", 329e-12),   # JJ as inductor
        ("C1", "2", "0", 100e-15),    # Shunt capacitor
        ("R2", "2", "0", 50.0),
    ]
    
    # Circuit 2: Single LC stage with Taylor
    circuit_taylor = [
        ("P1", "1", "0", 1),
        ("R1", "1", "0", 50.0),
        ("NL1", "1", "2", "poly 329e-12, 0.0, 0.5"),  # Taylor as inductor
        ("C1", "2", "0", 100e-15),    # Shunt capacitor
        ("R2", "2", "0", 50.0),
    ]
    
    circuitdefs = Dict()
    wp = 2*pi*10e9
    
    # Test at high pump currents: 20%, 40%, 60%, 80% of Ic
    currents = [0.2e-6, 0.4e-6, 0.6e-6, 0.8e-6]
    
    results = []
    for Ip in currents
        result_dict = Dict{String,Any}("Ip_percent" => Ip * 1e6 * 100)
        try
            result_jj = hbnlsolve((wp,), (8,), [(mode=(1,), port=1, current=Ip)], 
                                 circuit_jj, circuitdefs; dc=false, odd=true, even=false,
                                 keyedarrays=Val(false), iterations=100)
            
            result_taylor = hbnlsolve((wp,), (8,), [(mode=(1,), port=1, current=Ip)], 
                                     circuit_taylor, circuitdefs; dc=false, odd=true, even=false,
                                     keyedarrays=Val(false), iterations=100)
            
            branch_flux_jj = result_jj.Rbnm * result_jj.nodeflux
            branch_flux_taylor = result_taylor.Rbnm * result_taylor.nodeflux
            
            nl_branch = 2
            Nmodes = length(result_jj.frequencies.modes)
            
            # Get fundamental flux
            flux_jj_fund = branch_flux_jj[(nl_branch-1)*Nmodes + 1]
            flux_taylor_fund = branch_flux_taylor[(nl_branch-1)*Nmodes + 1]
            
            # Check if we're in the nonlinear regime
            phi_jj = abs(flux_jj_fund)
            sin_exact = sin(phi_jj)
            taylor_3rd = phi_jj - phi_jj^3/6
            
            result_dict["converged"] = true
            result_dict["flux_jj"] = flux_jj_fund
            result_dict["flux_taylor"] = flux_taylor_fund
            result_dict["phi_jj"] = phi_jj
            result_dict["phi_taylor"] = abs(flux_taylor_fund)
            result_dict["fund_diff_percent"] = 100 * abs(phi_jj - abs(flux_taylor_fund)) / phi_jj
            result_dict["sin_exact"] = sin_exact
            result_dict["taylor_3rd"] = taylor_3rd
            result_dict["taylor_error"] = abs(sin_exact - taylor_3rd) / abs(sin_exact + 1e-10) * 100
        catch e
            result_dict["converged"] = false
            result_dict["error"] = sprint(showerror, e)
        end
        push!(results, result_dict)
    end
    
    # Check above critical current
    high_current_jj = Dict{String,Any}()
    high_current_taylor = Dict{String,Any}()
    
    try
        Ip_high = 1.2e-6  # 120% of Ic
        result_jj_high = hbnlsolve((wp,), (4,), [(mode=(1,), port=1, current=Ip_high)], 
                                  circuit_jj, circuitdefs; dc=false, iterations=50,
                                  keyedarrays=Val(false))
        high_current_jj["converged"] = true
        branch_flux = result_jj_high.Rbnm * result_jj_high.nodeflux
        high_current_jj["flux_magnitude"] = abs(branch_flux[3])  # branch 2, mode 1
    catch e
        high_current_jj["converged"] = false
        high_current_jj["error"] = sprint(showerror, e)
    end
    
    try
        Ip_high = 1.2e-6
        result_taylor_high = hbnlsolve((wp,), (4,), [(mode=(1,), port=1, current=Ip_high)], 
                                      circuit_taylor, circuitdefs; dc=false, iterations=50,
                                      keyedarrays=Val(false))
        high_current_taylor["converged"] = true
        branch_flux = result_taylor_high.Rbnm * result_taylor_high.nodeflux
        high_current_taylor["flux_magnitude"] = abs(branch_flux[3])
    catch e
        high_current_taylor["converged"] = false
        high_current_taylor["error"] = sprint(showerror, e)
    end
    
    Dict{String,Any}(
        "results" => results,
        "high_current_jj" => high_current_jj,
        "high_current_taylor" => high_current_taylor
    )
""")

print("="*60)
print("HIGH PUMP CURRENT TEST")
print("="*60)
print("Testing at large fractions of critical current (Ic = 1 μA)")
print("-"*80)
print(f"{'Ip (%Ic)':<10} {'Converged':<12} {'φ JJ':<12} {'φ Taylor':<12} {'Diff %':<10} {'Taylor Err %':<12}")
print("-"*80)

for r in result['results']:
    if r.get('converged', False):
        print(f"{r['Ip_percent']:<10.0f} {'Yes':<12} {r['phi_jj']:<12.4f} {r['phi_taylor']:<12.4f} "
              f"{r['fund_diff_percent']:<10.2f} {r['taylor_error']:<12.2f}")
    else:
        print(f"{r['Ip_percent']:<10.0f} {'No':<12} {'---':<12} {'---':<12} {'---':<10} {'---':<12}")
        if 'error' in r:
            print(f"  Error: {r['error'][:80]}...")

print(f"\n{'='*60}")
print("ABOVE CRITICAL CURRENT (120% of Ic)")
print(f"{'='*60}")
print(f"JJ converged: {result['high_current_jj'].get('converged', False)}")
if result['high_current_jj'].get('converged', False):
    print(f"  Flux magnitude: {result['high_current_jj'].get('flux_magnitude', 'N/A'):.4f}")
else:
    print(f"  Error: {result['high_current_jj'].get('error', 'Unknown')[:80]}...")
    
print(f"\nTaylor converged: {result['high_current_taylor'].get('converged', False)}")
if result['high_current_taylor'].get('converged', False):
    print(f"  Flux magnitude: {result['high_current_taylor'].get('flux_magnitude', 'N/A'):.4f}")
else:
    print(f"  Error: {result['high_current_taylor'].get('error', 'Unknown')[:80]}...")

# Show the nonlinearity for converged results
print(f"\n{'='*60}")
print("NONLINEARITY CHECK")
print(f"{'='*60}")
for r in result['results']:
    if r.get('converged', False):
        print(f"\nAt {r['Ip_percent']:.0f}% Ic:")
        print(f"  Normalized flux φ = {r['phi_jj']:.4f} rad")
        print(f"  sin(φ) = {r['sin_exact']:.4f}")
        print(f"  Taylor approx = {r['taylor_3rd']:.4f}")
        print(f"  Taylor error = {r['taylor_error']:.1f}%")

✅ Forced Revise update complete - changes should be reflected now
✅ Also triggered precompilation of JosephsonCircuits fork
HIGH PUMP CURRENT TEST
Testing at large fractions of critical current (Ic = 1 μA)
--------------------------------------------------------------------------------
Ip (%Ic)   Converged    φ JJ         φ Taylor     Diff %     Taylor Err %
--------------------------------------------------------------------------------
20         Yes          0.1085       0.1050       3.18       0.00        
40         Yes          0.2210       0.2137       3.30       0.00        
60         Yes          0.3433       0.3314       3.47       0.01        
80         Yes          0.4870       0.4696       3.58       0.05        

ABOVE CRITICAL CURRENT (120% of Ic)
JJ converged: True
  Flux magnitude: 1.8944

Taylor converged: True
  Flux magnitude: 1.4312

NONLINEARITY CHECK

At 20% Ic:
  Normalized flux φ = 0.1085 rad
  sin(φ) = 0.1082
  Taylor approx = 0.1082
  Taylor error = 0.0%

A

In [4]:
julia_setup.force_revise_update(jl)
jl.eval("JosephsonCircuits.clear_debug_log()")
result = jl.eval("""
    using JosephsonCircuits
    
    # Circuit definitions
    circuit_jj_jj = [
        ("P1", "1", "0", 1),
        ("R1", "1", "0", 50.0),
        ("Lj1", "1", "2", 329e-12),   # JJ as first inductor
        ("C1", "2", "0", 100e-15),    # First shunt capacitor
        ("Lj2", "2", "3", 329e-12),   # JJ as second inductor
        ("C2", "3", "0", 100e-15),    # Second shunt capacitor
        ("R2", "3", "0", 50.0),       # Load resistor
    ]
    
    circuit_jj_taylor = [
        ("P1", "1", "0", 1),
        ("R1", "1", "0", 50.0),
        ("Lj1", "1", "2", 329e-12),   # JJ as first inductor
        ("C1", "2", "0", 100e-15),    # First shunt capacitor
        ("NL1", "2", "3", "poly 329e-12, 0.0, 0.5"),  # Taylor as second inductor
        ("C2", "3", "0", 100e-15),    # Second shunt capacitor
        ("R2", "3", "0", 50.0),       # Load resistor
    ]
    
    
    circuit_taylor_taylor = [
        ("P1", "1", "0", 1),
        ("R1", "1", "0", 50.0),
        ("NL1", "1", "2", "poly 329e-12, 0.0, 0.5"),  # Taylor as first inductor
        ("C1", "2", "0", 100e-15),    # First shunt capacitor
        ("NL2", "2", "3", "poly 329e-12, 0.0, 0.5"),  # Taylor as second inductor
        ("C2", "3", "0", 100e-15),    # Second shunt capacitor
        ("R2", "3", "0", 50.0),       # Load resistor
    ]
    
    circuitdefs = Dict()
    wp = 2*pi*10e9
    
    # Test at different pump currents
    currents = [0.05e-6, 0.1e-6, 0.2e-6]
    
    results = []
    for Ip in currents
        result_dict = Dict{String,Any}("Ip_nA" => Ip * 1e9)
        
        try
            # Solve all three circuits
            result_jj_jj = hbnlsolve((wp,), (8,), [(mode=(1,), port=1, current=Ip)], 
                                  circuit_jj_jj, circuitdefs; dc=false, odd=true, even=false,
                                  keyedarrays=Val(false), iterations=100)
            
            result_jj_taylor = hbnlsolve((wp,), (8,), [(mode=(1,), port=1, current=Ip)], 
                                    circuit_jj_taylor, circuitdefs; dc=false, odd=true, even=false,
                                    keyedarrays=Val(false), iterations=100)
            
            result_taylor_taylor = hbnlsolve((wp,), (8,), [(mode=(1,), port=1, current=Ip)], 
                                      circuit_taylor_taylor, circuitdefs; dc=false, odd=true, even=false,
                                      keyedarrays=Val(false), iterations=100)
            
            # Get branch fluxes
            branch_flux_jj_jj = result_jj_jj.Rbnm * result_jj_jj.nodeflux
            branch_flux_jj_taylor = result_jj_taylor.Rbnm * result_jj_taylor.nodeflux
            branch_flux_taylor_taylor = result_taylor_taylor.Rbnm * result_taylor_taylor.nodeflux
            
            # Get info about nonlinear elements
            nl_elements_jj_jj = result_jj_jj.nonlinear_elements
            nl_elements_jj_taylor = result_jj_taylor.nonlinear_elements
            nl_elements_taylor_taylor = result_taylor_taylor.nonlinear_elements
            
            # Extract fluxes at nonlinear elements
            Nmodes = length(result_jj_jj.frequencies.modes)
            
            # For JJ-JJ circuit
            jj_branches = sort([result_jj_jj.Ljb.nzind...])
            flux_jj_jj_1 = abs(branch_flux_jj_jj[(jj_branches[1]-1)*Nmodes + 1])
            flux_jj_jj_2 = abs(branch_flux_jj_jj[(jj_branches[2]-1)*Nmodes + 1])
            
            # For JJ-Taylor circuit
            jj_branch = result_jj_taylor.Ljb.nzind[1]
            taylor_branches_mixed = [k for (k,v) in nl_elements_jj_taylor if v.type == :taylor]
            taylor_branch = taylor_branches_mixed[1]
            flux_jj_taylor_jj = abs(branch_flux_jj_taylor[(jj_branch-1)*Nmodes + 1])
            flux_jj_taylor_taylor = abs(branch_flux_jj_taylor[(taylor_branch-1)*Nmodes + 1])
            
            # For Taylor-Taylor circuit
            taylor_branches = sort(collect(keys(nl_elements_taylor_taylor)))
            flux_taylor_taylor_1 = abs(branch_flux_taylor_taylor[(taylor_branches[1]-1)*Nmodes + 1])
            flux_taylor_taylor_2 = abs(branch_flux_taylor_taylor[(taylor_branches[2]-1)*Nmodes + 1])
            
            result_dict["converged"] = true
            result_dict["flux_jj_jj_1"] = flux_jj_jj_1
            result_dict["flux_jj_jj_2"] = flux_jj_jj_2
            result_dict["flux_jj_taylor_jj"] = flux_jj_taylor_jj
            result_dict["flux_jj_taylor_taylor"] = flux_jj_taylor_taylor
            result_dict["flux_taylor_taylor_1"] = flux_taylor_taylor_1
            result_dict["flux_taylor_taylor_2"] = flux_taylor_taylor_2
            
        catch e
            result_dict["converged"] = false
            result_dict["error"] = sprint(showerror, e)
        end
        
        push!(results, result_dict)
    end
    
    Dict{String,Any}("results" => results)
""")

print("="*60)
print("TWO-STAGE LC LADDER TEST (with load)")
print("="*60)
print("Testing two-stage LC ladder with nonlinear inductors and load termination")
print("-"*100)
print(f"{'Ip (nA)':<10} {'JJ-JJ Circuit':<25} {'JJ-Taylor Circuit':<30} {'Taylor-Taylor Circuit':<25}")
print(f"{'':10} {'φ1':<12} {'φ2':<12} {'φ_JJ':<12} {'φ_Taylor':<12} {'φ1':<12} {'φ2':<12}")
print("-"*100)

for r in result['results']:
    if r.get('converged', False):
        print(f"{r['Ip_nA']:<10.0f} "
              f"{r['flux_jj_jj_1']:<12.4f} {r['flux_jj_jj_2']:<12.4f} "
              f"{r['flux_jj_taylor_jj']:<12.4f} {r['flux_jj_taylor_taylor']:<12.4f} "
              f"{r['flux_taylor_taylor_1']:<12.4f} {r['flux_taylor_taylor_2']:<12.4f}")
    else:
        print(f"{r['Ip_nA']:<10.0f} {'Failed to converge'}")
        if 'error' in r:
            print(f"  Error: {r['error'][:80]}...")



# Get debug messages
print("\nDebug messages:")
debug_msgs = jl.eval("JosephsonCircuits.get_debug_log()")
for msg in debug_msgs:
    print(f"  {msg}")

✅ Forced Revise update complete - changes should be reflected now
✅ Also triggered precompilation of JosephsonCircuits fork
TWO-STAGE LC LADDER TEST (with load)
Testing two-stage LC ladder with nonlinear inductors and load termination
----------------------------------------------------------------------------------------------------
Ip (nA)    JJ-JJ Circuit             JJ-Taylor Circuit              Taylor-Taylor Circuit    
           φ1           φ2           φ_JJ         φ_Taylor     φ1           φ2          
----------------------------------------------------------------------------------------------------
50         0.0259       0.0259       0.0259       0.0259       0.0259       0.0259      
100        0.0519       0.0519       0.0519       0.0519       0.0519       0.0519      
200        0.1042       0.1041       0.1042       0.1041       0.1042       0.1041      

Debug messages:


In [5]:
print("\nComparison L-L, JJ-JJ and NL-NL at low currents (in the linear regime):")
julia_setup.force_revise_update(jl)
jl.eval("JosephsonCircuits.clear_debug_log()")
result = jl.eval(r"""
    using JosephsonCircuits
    using LinearAlgebra: norm
                 
    io = IOBuffer()
    
    # Circuit definitions
    circuit_l_l = [
        ("P1", "1", "0", 1),
        ("R1", "1", "0", 50.0),
        ("L1", "1", "2", 329e-12),
        ("C1", "2", "0", 132e-15),
        ("L2", "2", "3", 329e-12),
        ("C2", "3", "0", 132e-15),
        ("R2", "3", "0", 50.0),
    ]

    circuit_jj_jj = [
        ("P1", "1", "0", 1),
        ("R1", "1", "0", 50.0),
        ("Lj1", "1", "2", 329e-12),
        ("C1", "2", "0", 132e-15),
        ("Lj2", "2", "3", 329e-12),
        ("C2", "3", "0", 132e-15),
        ("R2", "3", "0", 50.0),
    ]
    
    circuit_nl_nl = [
        ("P1", "1", "0", 1),
        ("R1", "1", "0", 50.0),
        ("NL1", "1", "2", "poly 329e-12, 0.0, 0.5"),
        ("C1", "2", "0", 132e-15),
        ("NL2", "2", "3", "poly 329e-12, 0.0, 0.5"),
        ("C2", "3", "0", 132e-15),
        ("R2", "3", "0", 50.0),
    ]
    
    circuitdefs = Dict()
    wp = 2*pi*10e9
    Ip = 0.01e-6  # 0.01 uA
    
    results = Dict()
                 
    # Step 1: Solve L-L circuit
    JosephsonCircuits.debug_log("=== Step 1: Solving L-L circuit ===")
    result_l = hbnlsolve((wp,), (4,), [(mode=(1,), port=1, current=Ip)], 
                         circuit_l_l, circuitdefs; dc=false, iterations=100,
                         keyedarrays=Val(false))
    
    # Get the solution
    l_nodeflux = result_l.nodeflux
    branch_flux_l = result_l.Rbnm * l_nodeflux
    Nmodes = result_l.Nmodes
    
    # Report l fluxes
    JosephsonCircuits.debug_log("L-L solution found:")
    for branch in 1:3
        flux_mag = abs(branch_flux_l[(branch-1)*Nmodes + 1])
        println(io, "  Branch $branch: |flux| = $flux_mag")
    end
    
    # Step 2: Solve JJ-JJ circuit
    println(io, "=== Step 2: Solving JJ-JJ circuit ===")
    result_jj = hbnlsolve((wp,), (4,), [(mode=(1,), port=1, current=Ip)], 
                         circuit_jj_jj, circuitdefs; dc=false, iterations=100,
                         keyedarrays=Val(false))
    
    # Get the solution
    jj_nodeflux = result_jj.nodeflux
    branch_flux_jj = result_jj.Rbnm * jj_nodeflux
    Nmodes = result_jj.Nmodes
    
    # Report JJ fluxes
    JosephsonCircuits.debug_log("JJ-JJ solution found:")
    for branch in 1:3
        flux_mag = abs(branch_flux_jj[(branch-1)*Nmodes + 1])
        JosephsonCircuits.debug_log("  Branch $branch: |flux| = $flux_mag")
    end
    
    # Step 3: Solve NL-NL with zero initial guess (default)
    JosephsonCircuits.debug_log("\\n=== Step 3: Solving NL-NL with zero initial guess ===")
    try
        result_nl_zero = hbnlsolve((wp,), (4,), [(mode=(1,), port=1, current=Ip)], 
                                  circuit_nl_nl, circuitdefs; dc=false, iterations=100,
                                  x0=nothing, keyedarrays=Val(false))
        
        branch_flux_nl_zero = result_nl_zero.Rbnm * result_nl_zero.nodeflux
        
        JosephsonCircuits.debug_log("NL-NL with zero initial guess:")
        for branch in 1:3
            flux_mag = abs(branch_flux_nl_zero[(branch-1)*Nmodes + 1])
            JosephsonCircuits.debug_log("  Branch $branch: |flux| = $flux_mag")
        end
        
        results["nl_zero_success"] = true
        results["nl_zero_fluxes"] = [abs(branch_flux_nl_zero[(b-1)*Nmodes + 1]) for b in 1:3]
    catch e
        JosephsonCircuits.debug_log("  Failed: $(sprint(showerror, e))")
        results["nl_zero_success"] = false
    end        
                 
    # Store L results for comparison
    results["l_fluxes"] = [abs(branch_flux_l[(b-1)*Nmodes + 1]) for b in 1:3]             
    
    # Store JJ results for comparison
    results["jj_fluxes"] = [abs(branch_flux_jj[(b-1)*Nmodes + 1]) for b in 1:3]
    
    results
""")

# Get debug messages
debug_msgs = jl.eval("JosephsonCircuits.get_debug_log()")

print("="*60)
print("COMPARISON")
print("="*60)
print(f"{'Configuration':<20} {'Branch 1 |φ|':<15} {'Branch 2 |φ|':<15} {'Branch 3 |φ|':<15}")
print("-"*80)

# L-L reference
if 'l_fluxes' in result:
    fluxes = result['l_fluxes']
    print(f"{'L-L (reference)':<20} {fluxes[0]:<15.6f} {fluxes[1]:<15.6f} {fluxes[2]:<15.6f}")

# JJ-JJ reference
if 'jj_fluxes' in result:
    fluxes = result['jj_fluxes']
    print(f"{'JJ-JJ':<20} {fluxes[0]:<15.6f} {fluxes[1]:<15.6f} {fluxes[2]:<15.6f}")

# NL-NL with zero initial guess
if result.get('nl_zero_success', False):
    fluxes = result['nl_zero_fluxes']
    print(f"{'NL-NL':<20} {fluxes[0]:<15.6f} {fluxes[1]:<15.6f} {fluxes[2]:<15.6f}")
else:
    print(f"{'NL-NL':<20} {'FAILED':<15} {'---':<15} {'---':<15}")


print("\nDebug output:")
for msg in debug_msgs:
    print(f"  {msg}")


Comparison L-L, JJ-JJ and NL-NL at low currents (in the linear regime):
✅ Forced Revise update complete - changes should be reflected now
✅ Also triggered precompilation of JosephsonCircuits fork
COMPARISON
Configuration        Branch 1 |φ|    Branch 2 |φ|    Branch 3 |φ|   
--------------------------------------------------------------------------------
L-L (reference)      0.010926        0.005546        0.005346       
JJ-JJ                0.010926        0.005546        0.005346       
NL-NL                0.010926        0.005546        0.005346       

Debug output:
  === Step 1: Solving L-L circuit ===
  L-L solution found:
  JJ-JJ solution found:
    Branch 1: |flux| = 0.010926080568800314
    Branch 2: |flux| = 0.005546346781179312
    Branch 3: |flux| = 0.005345925022359958
  \n=== Step 3: Solving NL-NL with zero initial guess ===
  NL-NL with zero initial guess:
    Branch 1: |flux| = 0.010926080569047194
    Branch 2: |flux| = 0.005546346781548587
    Branch 3: |flux| = 0.

In [6]:
print("\nTesting hblinsolve with proper two-port circuits:")
julia_setup.force_revise_update(jl)
jl.eval("JosephsonCircuits.clear_debug_log()")
result = jl.eval(r"""
    using JosephsonCircuits
    
    # Modified circuits with Port 2 added
    circuit_l_l = [
        ("P1", "1", "0", 1),
        ("R1", "1", "0", 50.0),
        ("L1", "1", "2", 329e-12),
        ("C1", "2", "0", 132e-15),
        ("L2", "2", "3", 329e-12),
        ("C2", "3", "0", 132e-15),
        ("P2", "3", "0", 2),  # Added Port 2
        ("R2", "3", "0", 50.0),
    ]

    circuit_jj_jj = [
        ("P1", "1", "0", 1),
        ("R1", "1", "0", 50.0),
        ("Lj1", "1", "2", 329e-12),
        ("C1", "2", "0", 132e-15),
        ("Lj2", "2", "3", 329e-12),
        ("C2", "3", "0", 132e-15),
        ("P2", "3", "0", 2),  # Added Port 2
        ("R2", "3", "0", 50.0),
    ]
    
    circuit_nl_nl = [
        ("P1", "1", "0", 1),
        ("R1", "1", "0", 50.0),
        ("NL1", "1", "2", "poly 329e-12, 0.0, 0.5"),
        ("C1", "2", "0", 132e-15),
        ("NL2", "2", "3", "poly 329e-12, 0.0, 0.5"),
        ("C2", "3", "0", 132e-15),
        ("P2", "3", "0", 2),  # Added Port 2
        ("R2", "3", "0", 50.0),
    ]
    
    circuitdefs = Dict()
    ws = 2*pi*10e9  # Signal frequency
    
    results = Dict()
    
    # Step 1: Test L-L circuit (pure linear analysis)
    JosephsonCircuits.debug_log("=== Testing hblinsolve with L-L circuit (no pump) ===")
    try
        lin_result = hblinsolve(ws, circuit_l_l, circuitdefs; 
                               Nmodulationharmonics=(1,), keyedarrays=Val(false))
        
        # Extract S-parameters
        if hasfield(typeof(lin_result), :S) && size(lin_result.S, 1) >= 2
            S11 = lin_result.S[1,1,1]
            S21 = lin_result.S[2,1,1]
            S12 = lin_result.S[1,2,1]
            S22 = lin_result.S[2,2,1]
            
            results["l_l_S11"] = abs(S11)
            results["l_l_S21"] = abs(S21)
            results["l_l_S21_dB"] = 20*log10(abs(S21))
            
            JosephsonCircuits.debug_log("L-L S-parameters:")
            JosephsonCircuits.debug_log("  S11: $(abs(S11)) ($(20*log10(abs(S11))) dB)")
            JosephsonCircuits.debug_log("  S21: $(abs(S21)) ($(20*log10(abs(S21))) dB)")
            JosephsonCircuits.debug_log("  S12: $(abs(S12)) ($(20*log10(abs(S12))) dB)")
            JosephsonCircuits.debug_log("  S22: $(abs(S22)) ($(20*log10(abs(S22))) dB)")
        else
            JosephsonCircuits.debug_log("Could not extract S-parameters")
        end
        results["l_l_success"] = true
    catch e
        JosephsonCircuits.debug_log("L-L failed: $(sprint(showerror, e))")
        results["l_l_success"] = false
    end
    
    results
""")

# Get debug messages
debug_msgs = jl.eval("JosephsonCircuits.get_debug_log()")

print("="*60)
print("HBLINSOLVE TEST RESULTS")
print("="*60)

if result.get('l_l_success', False):
    print(f"L-L circuit: SUCCESS")
    if 'l_l_S21' in result:
        print(f"  S11: {result.get('l_l_S11', 'N/A'):.6f}")
        print(f"  S21: {result['l_l_S21']:.6f} ({result.get('l_l_S21_dB', 'N/A'):.2f} dB)")
else:
    print(f"L-L circuit: FAILED")

print("\nDebug output:")
for msg in debug_msgs:
    print(f"  {msg}")


Testing hblinsolve with proper two-port circuits:
✅ Forced Revise update complete - changes should be reflected now
✅ Also triggered precompilation of JosephsonCircuits fork
HBLINSOLVE TEST RESULTS
L-L circuit: SUCCESS
  S11: 0.154863
  S21: 0.987936 (-0.11 dB)

Debug output:
  === Testing hblinsolve with L-L circuit (no pump) ===
  L-L S-parameters:
    S11: 0.1548629798748622 (-16.201047765874602 dB)
    S21: 0.9879359581796171 (-0.1054241428761245 dB)
    S12: 0.9879359581796172 (-0.10542414287612353 dB)
    S22: 0.15486297987486228 (-16.2010477658746 dB)


In [7]:
print("\nTesting hbsolve (combined solver):")
julia_setup.force_revise_update(jl)
jl.eval("JosephsonCircuits.clear_debug_log()")
result = jl.eval(r"""
    using JosephsonCircuits
    
    # Test circuits
    circuit_jj = [
        ("P1", "1", "0", 1),
        ("R1", "1", "0", 50.0),
        ("Lj1", "1", "2", 329e-12),
        ("C1", "2", "0", 132e-15),
        ("Lj2", "2", "3", 329e-12),
        ("C2", "3", "0", 132e-15),
        ("P2", "3", "0", 2),
        ("R2", "3", "0", 50.0),
    ]
    
    circuit_nl = [
        ("P1", "1", "0", 1),
        ("R1", "1", "0", 50.0),
        ("NL1", "1", "2", "poly 329e-12, 0.0, 0.5"),
        ("C1", "2", "0", 132e-15),
        ("NL2", "2", "3", "poly 329e-12, 0.0, 0.5"),
        ("C2", "3", "0", 132e-15),
        ("P2", "3", "0", 2),
        ("R2", "3", "0", 50.0),
    ]
    
    # Use numeric circuit definitions - no symbolic variables
    circuitdefs = Dict{Symbol, Float64}()
    
    wp = (2*pi*10e9,)  # Pump frequency as tuple
    ws = 2*pi*10.1e9   # Signal frequency
    
    results = Dict()
    
    # Test at different pump powers
    pump_currents = [0.1e-6, 0.3e-6, 0.5e-6]
    
    for (i, Ip) in enumerate(pump_currents)
        JosephsonCircuits.debug_log("\\n=== Testing hbsolve with Ip = $(Ip*1e6) μA ===")
        
        # Create sources array
        sources = [(mode=(1,), port=1, current=Ip)]
        
        # Test JJ circuit
        try
            # Call hbsolve with new syntax
            hb_result_jj = hbsolve(
                ws,                    # Signal frequency
                wp,                    # Pump frequency tuple
                sources,               # Sources array
                (1,),                  # Modulation harmonics tuple (signal only)
                (8,),                  # Pump harmonics tuple
                circuit_jj,
                circuitdefs;
                dc=false,
                threewavemixing=false,
                fourwavemixing=true,
                keyedarrays=Val(false)
            )
            
            # Extract gain
            S21 = hb_result_jj.linearized.S[2,1,1]
            gain_dB = 20*log10(abs(S21))
            results["jj_gain_$(i)"] = gain_dB
            results["jj_success_$(i)"] = true
            JosephsonCircuits.debug_log("  JJ SUCCESS: Gain = $(gain_dB) dB")
            
        catch e
            JosephsonCircuits.debug_log("  JJ FAILED: $(sprint(showerror, e))")
            results["jj_success_$(i)"] = false
        end
        
        # Test NL circuit
        try
            hb_result_nl = hbsolve(
                ws,
                wp,
                sources,
                (1,),
                (8,),
                circuit_nl,
                circuitdefs;
                dc=false,
                threewavemixing=false,
                fourwavemixing=true,
                keyedarrays=Val(false)
            )
            
            S21 = hb_result_nl.linearized.S[2,1,1]
            gain_dB = 20*log10(abs(S21))
            results["nl_gain_$(i)"] = gain_dB
            results["nl_success_$(i)"] = true
            JosephsonCircuits.debug_log("  NL SUCCESS: Gain = $(gain_dB) dB")
            
        catch e
            JosephsonCircuits.debug_log("  NL FAILED: $(sprint(showerror, e))")
            results["nl_success_$(i)"] = false
        end
    end
    
    # Test with DC bias
    JosephsonCircuits.debug_log("\\n=== Testing hbsolve with DC bias ===")
    Ip = 0.3e-6
    Idc = 0.05e-6
    
    try
        # Sources with DC and pump
        sources_dc = [
            (mode=(0,), port=1, current=Idc),   # DC bias
            (mode=(1,), port=1, current=Ip)     # Pump
        ]
        
        hb_result_dc = hbsolve(
            ws,
            wp,
            sources_dc,
            (1,),
            (8,),
            circuit_jj,
            circuitdefs;
            dc=true,              # Enable DC
            threewavemixing=false,
            fourwavemixing=true,
            keyedarrays=Val(false)
        )
        
        S21 = hb_result_dc.linearized.S[2,1,1]
        gain_dB = 20*log10(abs(S21))
        results["dc_gain"] = gain_dB
        results["dc_test_success"] = true
        JosephsonCircuits.debug_log("  DC test SUCCESS: Gain = $(gain_dB) dB")
        
    catch e
        JosephsonCircuits.debug_log("  DC test FAILED: $(sprint(showerror, e))")
        results["dc_test_success"] = false
    end
    
    # Test with signal+idler
    JosephsonCircuits.debug_log("\\n=== Testing hbsolve with signal+idler ===")
    
    try
        sources = [(mode=(1,), port=1, current=0.3e-6)]
        
        hb_result_idler = hbsolve(
            ws,
            wp,
            sources,
            (2,),                 # Signal + idler
            (8,),
            circuit_jj,
            circuitdefs;
            dc=false,
            threewavemixing=false,
            fourwavemixing=true,
            keyedarrays=Val(false)
        )
        
        results["idler_smatrix_size"] = size(hb_result_idler.linearized.S)
        results["idler_test_success"] = true
        JosephsonCircuits.debug_log("  Signal+idler SUCCESS: S-matrix size = $(size(hb_result_idler.linearized.S))")
        
    catch e
        JosephsonCircuits.debug_log("  Signal+idler FAILED: $(sprint(showerror, e))")
        results["idler_test_success"] = false
    end
    
    results
""")

# Get debug messages
debug_msgs = jl.eval("JosephsonCircuits.get_debug_log()")

print("="*60)
print("HBSOLVE TEST RESULTS")
print("="*60)
print(f"{'Pump (μA)':<12} {'JJ Status':<15} {'JJ Gain (dB)':<15} {'NL Status':<15} {'NL Gain (dB)':<15}")
print("-"*60)

pump_currents = [0.1, 0.3, 0.5]
for i, Ip in enumerate(pump_currents, 1):
    jj_status = "SUCCESS" if result.get(f'jj_success_{i}', False) else "FAILED"
    jj_gain = result.get(f'jj_gain_{i}', float('nan'))
    nl_status = "SUCCESS" if result.get(f'nl_success_{i}', False) else "FAILED"
    nl_gain = result.get(f'nl_gain_{i}', float('nan'))
    
    print(f"{Ip:<12.1f} {jj_status:<15} {jj_gain:<15.2f} {nl_status:<15} {nl_gain:<15.2f}")

print(f"\nDC bias test: {'SUCCESS' if result.get('dc_test_success', False) else 'FAILED'}")
if 'dc_gain' in result:
    print(f"  Gain with DC: {result['dc_gain']:.2f} dB")

print(f"\nSignal+Idler test: {'SUCCESS' if result.get('idler_test_success', False) else 'FAILED'}")
if 'idler_smatrix_size' in result:
    print(f"  S-matrix size: {result['idler_smatrix_size']}")

print("\nDebug output:")
for msg in debug_msgs:
    print(f"  {msg}")


Testing hbsolve (combined solver):
✅ Forced Revise update complete - changes should be reflected now
✅ Also triggered precompilation of JosephsonCircuits fork
HBSOLVE TEST RESULTS
Pump (μA)    JJ Status       JJ Gain (dB)    NL Status       NL Gain (dB)   
------------------------------------------------------------
0.1          SUCCESS         -0.11           SUCCESS         -0.11          
0.3          SUCCESS         -0.12           SUCCESS         -0.12          
0.5          SUCCESS         -0.13           SUCCESS         -0.13          

DC bias test: FAILED

Signal+Idler test: SUCCESS
  S-matrix size: (6, 6, 1)

Debug output:
  \n=== Testing hbsolve with Ip = 0.09999999999999999 μA ===
    JJ SUCCESS: Gain = -0.10984581808190871 dB
    NL SUCCESS: Gain = -0.10984629322726006 dB
  \n=== Testing hbsolve with Ip = 0.3 μA ===
    JJ SUCCESS: Gain = -0.11536027395222093 dB
    NL SUCCESS: Gain = -0.11541001223608921 dB
  \n=== Testing hbsolve with Ip = 0.5 μA ===
    JJ SUCCESS: Gai

In [8]:
print("\nTesting Simple JPA test comparing JJ and NL")

if set_fork:
    julia_setup.force_revise_update(jl)
    jl.eval("JosephsonCircuits.clear_debug_log()")

result = jl.eval(r"""
    using JosephsonCircuits
    using LinearAlgebra
    
    io = IOBuffer()
                     
    # Wrap everything in a function to avoid scope issues
    function test_jpa()  
                                  
        # Parameters
        ws = 2*pi*4.75*1e9  # Single frequency near pump
        wp = (2*pi*4.75001*1e9,)  # Pump frequency
        Ip = 0.00565e-6 # 5.65 nA
        Lj = 1000.0e-12
        
        sources = [
            (mode=(1,), port=1, current=Ip)
        ]
              
                    
        circuitdefs = Dict{Symbol,Complex{Float64}}(
            :Lj => Lj,
            :c1 => 0.0,             
            :c2 => 0.5             
        )                 
                 
        println(io,"=== Testing Simple JPA ===")
        println(io,"Pump: $(Ip*1e9) nA @ $(wp[1]/(2*pi*1e9)) GHz")
        println(io,"Signal: $(ws/(2*pi*1e9)) GHz")                    
                    
        ########################## JJ ############################
                    
        # Simple JPA circuit
        circuit_jj = [
            ("P1", "1", "0", 1),
            ("R1", "1", "0", 50.0),
            ("C1", "1", "2", 100.0e-15),
            ("Lj1", "2", "0", Lj),
            ("C2", "2", "0", 1000.0e-15)
        ]                                         
        
        # Run simulation
        hb_jj = hbsolve(
            ws, wp, sources, (8,), (16,),
            circuit_jj, circuitdefs;        
        )
                    
        # Extract the JJ solution
        jj_nodeflux = vec(hb_jj.nonlinear.nodeflux)
        println(io, "  JJ solution norm: $(norm(jj_nodeflux))")                 
                    
        ########################## NL ############################                                         
                    
        # Simple JPA circuit
        circuit_nl = [
            ("P1", "1", "0", 1),
            ("R1", "1", "0", 50.0),
            ("C1", "1", "2", 100.0e-15),
            ("NL1", "2", "0", "poly Lj, c1, c2"),
            ("C2", "2", "0", 1000.0e-15)
        ]                                         
        
        # Run simulation
        hb_nl = hbsolve(
            ws, wp, sources, (8,), (16,),
            circuit_nl, circuitdefs;        
        )
                    
        # Extract the JJ solution
        nl_nodeflux = vec(hb_nl.nonlinear.nodeflux)
        println(io, "  NL solution norm: $(norm(nl_nodeflux))") 

        
        S11_jj = hb_jj.linearized.S(
                outputmode=(0,), outputport=1,
                inputmode=(0,), inputport=1, freqindex=1
            )
                    
        S11_nl = hb_nl.linearized.S(
                outputmode=(0,), outputport=1,
                inputmode=(0,), inputport=1, freqindex=1
            )

        gain_jj = 10*log10(abs2(S11_jj))
        gain_nl = 10*log10(abs2(S11_nl))
                 
        return (gain_jj, gain_nl, String(take!(io)))    
    end
    
    # Call the function
    test_jpa()
""")

gain_jj, gain_nl, debug_output = result


print("="*60)
print("\nJPA TEST")
print("\n" + "="*60)
print(f"\nGain with JJ: {gain_jj:.2f} dB")
print(f"Gain with NL: {gain_nl:.2f} dB")
print("\nDebug output:")
print(debug_output)
if set_fork:
    debug_msgs = jl.eval("JosephsonCircuits.get_debug_log()")
    for msg in debug_msgs:
        print(f"  {msg}")


Testing Simple JPA test comparing JJ and NL
✅ Forced Revise update complete - changes should be reflected now
✅ Also triggered precompilation of JosephsonCircuits fork

JPA TEST


Gain with JJ: 13.30 dB
Gain with NL: 14.32 dB

Debug output:
=== Testing Simple JPA ===
Pump: 5.65 nA @ 4.75001 GHz
Signal: 4.749999999999999 GHz
  JJ solution norm: 0.14624146080969944
  NL solution norm: 0.1473128472293606



In [9]:
print("\nTesting SNAILPA DC bias test")

if set_fork:
    julia_setup.force_revise_update(jl)
    jl.eval("JosephsonCircuits.clear_debug_log()")

# Capture stdout
result = jl.eval(r"""
    using JosephsonCircuits
    using LinearAlgebra
    
    # Create a string buffer to capture output
    io = IOBuffer()
                 
    # Wrap everything in a function to avoid scope issues
    function test_snailpa()                
        
        # Parameters
        alpha = 0.29                                             
        Lj = 60e-12
        Lj_large = 60e-12 / alpha
        Cj = 10.0e-15
        Cj_large = Cj / alpha      
                 
        circuitdefs = Dict{Symbol,Complex{Float64}}(
            :Lj => Lj,
            :Lj_large => Lj_large,
        )                      
        
        # Parameters
        Ip = 4.4e-6
        Idc = 159e-6 * 0.94
        wp = (2*pi*16e9,)
        ws = 2*pi*8e9                 
                    
        sources = [
            (mode=(0,), port=2, current=Idc),
            (mode=(1,), port=2, current=Ip)
        ]                 
                            

        ########################## JJ ############################                     

        # SNAILPA circuit    
        circuit_jj = [
            ("P1", "1", "0", 1),
            ("R1", "1", "0", 50.0),
            ("L0", "1", "0", 100.0e-9),
            ("C1", "1", "2", 0.048e-12),
            ("L1", "2", "3", 0.4264e-9 * 1.25),
            ("C2", "2", "0", 0.4e-12 * 1.25),
            ("Lj1", "3", "0", Lj_large),
            ("Cj1", "3", "0", Cj_large),
            ("L2", "3", "4", 34e-12),
            ("Lj2", "4", "5", Lj),
            ("Cj2", "4", "5", Cj),
            ("Lj3", "5", "6", Lj),
            ("Cj3", "5", "6", Cj),
            ("Lj4", "6", "0", Lj),
            ("Cj4", "6", "0", Cj),
            ("L3", "7", "0", 0.74e-12),
            ("K1", "L2", "L3", 0.999),
            ("P2", "7", "0", 2),
            ("R2", "7", "0", 1000.0)
        ]

            
        hb_jj = hbsolve(
            ws, wp, sources, (8,), (16,),
            circuit_jj, circuitdefs;
            dc=true, threewavemixing=true, fourwavemixing=true          
        )                            
        
        # Extract the JJ solution
        jj_nodeflux = vec(hb_jj.nonlinear.nodeflux)
        println(io, "  JJ solution norm: $(norm(jj_nodeflux))")                 
                    
        ############################## NL ################################ 
        
        # SNAILPA circuit    
        circuit_nl = [
            ("P1", "1", "0", 1),
            ("R1", "1", "0", 50.0),
            ("L0", "1", "0", 100.0e-9),
            ("C1", "1", "2", 0.048e-12),
            ("L1", "2", "3", 0.4264e-9 * 1.25),
            ("C2", "2", "0", 0.4e-12 * 1.25),
            ("NL1", "3", "0", "poly Lj_large, 0.0, 0.5"),
            ("Cj1", "3", "0", Cj_large),
            ("L2", "3", "4", 34e-12),
            ("NL2", "4", "5", "poly Lj, 0.0, 0.5"),
            ("Cj2", "4", "5", Cj),
            ("NL3", "5", "6", "poly Lj, 0.0, 0.5"),
            ("Cj3", "5", "6", Cj),
            ("NL4", "6", "0", "poly Lj, 0.0, 0.5"),
            ("Cj4", "6", "0", Cj),
            ("L3", "7", "0", 0.74e-12),
            ("K1", "L2", "L3", 0.999),
            ("P2", "7", "0", 2),
            ("R2", "7", "0", 1000.0)
        ]

            
        hb_nl = hbsolve(
            ws, wp, sources, (8,), (16,),
            circuit_nl, circuitdefs;
            dc=true, threewavemixing=true, fourwavemixing=true                
        )               

        # Extract the NL solution
        nl_nodeflux = vec(hb_nl.nonlinear.nodeflux)
        println(io, "  NL solution norm: $(norm(nl_nodeflux))")                               
        
        ############################## gain ################################                 

        # Extract S parameters and calculate gains
        S11_jj = hb_jj.linearized.S(
            outputmode=(0,), outputport=1,
            inputmode=(0,), inputport=1, freqindex=1
        )
        S11_nl = hb_nl.linearized.S(
            outputmode=(0,), outputport=1,
            inputmode=(0,), inputport=1, freqindex=1
        )
        
        gain_jj = 10*log10(abs2(S11_jj))
        gain_nl = 10*log10(abs2(S11_nl))


        return (gain_jj, gain_nl, String(take!(io)))

    end
    
    # Call the function
    test_snailpa()                              

""")

gain_jj, gain_nl, debug_output = result
    

print("="*60)
print("\nSNAILPA DC BIAS TEST")
print("\n" + "="*60)
print(f"\nGain with JJ: {gain_jj:.2f} dB")
print(f"Gain with NL: {gain_nl:.2f} dB")
print("\nDebug output:")
print(debug_output)
if set_fork:
    debug_msgs = jl.eval("JosephsonCircuits.get_debug_log()")
    for msg in debug_msgs:
        print(f"  {msg}")



Testing SNAILPA DC bias test
✅ Forced Revise update complete - changes should be reflected now
✅ Also triggered precompilation of JosephsonCircuits fork

SNAILPA DC BIAS TEST


Gain with JJ: 0.20 dB
Gain with NL: 19.57 dB

Debug output:
  JJ solution norm: 2.1123033462965775
  NL solution norm: 2.1300288192242642



In [10]:
print("\nJTWPA (4WM) test comparing JJ and NL")

if set_fork:
    julia_setup.force_revise_update(jl)
    jl.eval("JosephsonCircuits.clear_debug_log()")

result = jl.eval(r"""
    using JosephsonCircuits
    using LinearAlgebra
                 
    io = IOBuffer()
    
    # Wrap everything in a function to avoid scope issues
    function test_jtwpa_transfer()
                 
        # Define as regular variables
        Rleft = 50.0
        Rright = 50.0
        Cg = 45.0e-15
        Cc = 30.0e-15
        Cr = 2.8153e-12
        Lr = 1.70e-10
        Cj = 55e-15
        
        # Common circuit parameters
        Nj=1000
        pmrpitch = 4
        circuitdefs = Dict(            
            :Cg => Cg,
            :Cc => Cc,
            :Cr =>  Cr,
            :Lr => Lr,
            :Cj => Cj,
            :Rleft => Rleft,
            :Rright => Rright,
        )
        
        # Test parameters
        ws = 2*pi*8.12*1e9
        wp = (2*pi*7.12*1e9,)
        Ip = 1.85e-6
        sources = [(mode=(1,),port=1,current=Ip)]
        Npumpharmonics = (20,)
        Nmodulationharmonics = (10,)
                 

        ########################## JJ ############################                 
        
        # First, build and solve with JJs
        circuit_jj = Tuple{String,String,String,Any}[]
        
        # port on the input side
        push!(circuit_jj,("P$(1)_$(0)","1","0",1))
        push!(circuit_jj,("R$(1)_$(0)","1","0",Rleft))
                
        #first half cap to ground
        push!(circuit_jj,("C$(1)_$(0)","1","0",Cg/2))
        
        #middle caps and jj's
        push!(circuit_jj,("Lj$(1)_$(2)","1","2",96.8e-12))                  
        push!(circuit_jj,("C$(1)_$(2)","1","2",Cj)) 
        
        j=2
        for i = 2:Nj-1
            if mod(i,pmrpitch) == pmrpitch÷2
                # make the jj cell with modified capacitance to ground
                push!(circuit_jj,("C$(j)_$(0)","$(j)","$(0)",Cg-Cc))
                push!(circuit_jj,("Lj$(j)_$(j+2)","$(j)","$(j+2)", 96.8e-12))                
                push!(circuit_jj,("C$(j)_$(j+2)","$(j)","$(j+2)",Cj))
                
                #make the pmr
                push!(circuit_jj,("C$(j)_$(j+1)","$(j)","$(j+1)",Cc))
                push!(circuit_jj,("C$(j+1)_$(0)","$(j+1)","$(0)",Cr))
                push!(circuit_jj,("L$(j+1)_$(0)","$(j+1)","$(0)",Lr))
                
                # increment the index
                j+=1
            else
                push!(circuit_jj,("C$(j)_$(0)","$(j)","$(0)",Cg))
                push!(circuit_jj,("Lj$(j)_$(j+1)","$(j)","$(j+1)", 96.8e-12))                
                push!(circuit_jj,("C$(j)_$(j+1)","$(j)","$(j+1)", Cj))
            end
            
            # increment the index
            j+=1
        end
        
        #last cap
        push!(circuit_jj,("C$(j)_$(0)","$(j)","$(0)",Cg/2))
        push!(circuit_jj,("R$(j)_$(0)","$(j)","$(0)",Rright))
        # port on the output side
        push!(circuit_jj,("P$(j)_$(0)","$(j)","$(0)",2))
                
        
        println(io, "1: Solving with Josephson junctions...")
        rpm_jj = hbsolve(
                ws, wp, sources, Nmodulationharmonics,
                Npumpharmonics, circuit_jj, circuitdefs;
        )
        
        # Extract the JJ solution
        jj_nodeflux = vec(rpm_jj.nonlinear.nodeflux)
        println(io, "  JJ solution norm: $(norm(jj_nodeflux))")
                 

        ########################## L ############################                 
        
        # First, build and solve with L
        circuit_l = Tuple{String,String,String,Any}[]
        
        # port on the input side
        push!(circuit_l,("P$(1)_$(0)","1","0",1))
        push!(circuit_l,("R$(1)_$(0)","1","0",Rleft))
                
        #first half cap to ground
        push!(circuit_l,("C$(1)_$(0)","1","0",Cg/2))
        
        #middle caps and l's
        push!(circuit_l,("L$(1)_$(2)","1","2",96.8e-12))                  
        push!(circuit_l,("C$(1)_$(2)","1","2",Cj)) 
        
        j=2
        for i = 2:Nj-1
            if mod(i,pmrpitch) == pmrpitch÷2
                # make the l cell with modified capacitance to ground
                push!(circuit_l,("C$(j)_$(0)","$(j)","$(0)",Cg-Cc))
                push!(circuit_l,("L$(j)_$(j+2)","$(j)","$(j+2)", 96.8e-12))                
                push!(circuit_l,("C$(j)_$(j+2)","$(j)","$(j+2)",Cj))
                
                #make the pmr
                push!(circuit_l,("C$(j)_$(j+1)","$(j)","$(j+1)",Cc))
                push!(circuit_l,("C$(j+1)_$(0)","$(j+1)","$(0)",Cr))
                push!(circuit_l,("L$(j+1)_$(0)","$(j+1)","$(0)",Lr))
                
                # increment the index
                j+=1
            else
                push!(circuit_l,("C$(j)_$(0)","$(j)","$(0)",Cg))
                push!(circuit_l,("L$(j)_$(j+1)","$(j)","$(j+1)",96.8e-12))                
                push!(circuit_l,("C$(j)_$(j+1)","$(j)","$(j+1)",Cj))
            end
            
            # increment the index
            j+=1
        end
        
        #last cap
        push!(circuit_l,("C$(j)_$(0)","$(j)","$(0)",Cg/2))
        push!(circuit_l,("R$(j)_$(0)","$(j)","$(0)",Rright))
        # port on the output side
        push!(circuit_l,("P$(j)_$(0)","$(j)","$(0)",2))
                
        
        println(io, "\n2: Solving with inductances...")
        rpm_l = hbsolve(
                ws, wp, sources, Nmodulationharmonics,
                Npumpharmonics, circuit_l, circuitdefs;
        )
        
        # Extract the L solution
        l_nodeflux = vec(rpm_l.nonlinear.nodeflux)
        println(io, "  L solution norm: $(norm(l_nodeflux))")


        ############################## NL ################################

        
        # Now build circuit with NL elements
        circuit_nl = Tuple{String,String,String,Any}[]
        
        # port on the input side
        push!(circuit_nl,("P$(1)_$(0)","1","0",1))
        push!(circuit_nl,("R$(1)_$(0)","1","0",Rleft))
                
        #first half cap to ground
        push!(circuit_nl,("C$(1)_$(0)","1","0",Cg/2))
        
        #middle caps and nl's        
        push!(circuit_nl,("NL$(1)_$(2)","1","2","poly 96.8e-12, 0.0, 0.5")) 
        push!(circuit_nl,("C$(1)_$(2)","1","2",Cj)) 
        
        j=2
        for i = 2:Nj-1
            if mod(i,pmrpitch) == pmrpitch÷2
                # make the nl cell with modified capacitance to ground
                push!(circuit_nl,("C$(j)_$(0)","$(j)","$(0)",Cg-Cc))                
                push!(circuit_nl,("NL$(j)_$(j+2)","$(j)","$(j+2)","poly 96.8e-12, 0.0, 0.5"))
                push!(circuit_nl,("C$(j)_$(j+2)","$(j)","$(j+2)",Cj))
                
                #make the pmr
                push!(circuit_nl,("C$(j)_$(j+1)","$(j)","$(j+1)",Cc))
                push!(circuit_nl,("C$(j+1)_$(0)","$(j+1)","$(0)",Cr))
                push!(circuit_nl,("L$(j+1)_$(0)","$(j+1)","$(0)",Lr))
                
                # increment the index
                j+=1
            else
                push!(circuit_nl,("C$(j)_$(0)","$(j)","$(0)",Cg))                
                push!(circuit_nl,("NL$(j)_$(j+1)","$(j)","$(j+1)","poly 96.8e-12, 0.0, 0.5"))
                push!(circuit_nl,("C$(j)_$(j+1)","$(j)","$(j+1)",Cj))
            end
            
            # increment the index
            j+=1
        end
        
        #last nl
        push!(circuit_nl,("C$(j)_$(0)","$(j)","$(0)",Cg/2))
        push!(circuit_nl,("R$(j)_$(0)","$(j)","$(0)",Rright))
        # port on the output side
        push!(circuit_nl,("P$(j)_$(0)","$(j)","$(0)",2))
        
        println(io, "\n3: Solving with NL using L solution as x0...")        
                 
        rpm_nl = hbsolve(
                ws, wp, sources, Nmodulationharmonics,
                Npumpharmonics, circuit_nl, circuitdefs;                         
        )
        
        # Compare the results
        nl_nodeflux = vec(rpm_nl.nonlinear.nodeflux)
        println(io, "  NL solution norm: $(norm(nl_nodeflux))")
        
        ############################## gain ################################                 

        # Extract S parameters and calculate gains
        S21_jj = rpm_jj.linearized.S(
            outputmode=(0,), outputport=2,
            inputmode=(0,), inputport=1, freqindex=1
        )
        S21_nl = rpm_nl.linearized.S(
            outputmode=(0,), outputport=2,
            inputmode=(0,), inputport=1, freqindex=1
        )
        
        gain_jj = 10*log10(abs2(S21_jj))
        gain_nl = 10*log10(abs2(S21_nl))


        return (gain_jj, gain_nl, String(take!(io)))
    end
    
    # Call the function
    test_jtwpa_transfer()
""")

gain_jj, gain_nl, debug_output = result

print("="*60)
print("\nJTWPA TEST: JJ solution -> NL with x0")
print("="*60)
print(f"\nGain with JJ: {gain_jj:.2f} dB")
print(f"Gain with NL: {gain_nl:.2f} dB")

print("\nDebug output:")
print(debug_output)

if set_fork:
    debug_msgs = jl.eval("JosephsonCircuits.get_debug_log()")
    for msg in debug_msgs:
        print(f"  {msg}")

    


JTWPA (4WM) test comparing JJ and NL
✅ Forced Revise update complete - changes should be reflected now
✅ Also triggered precompilation of JosephsonCircuits fork

JTWPA TEST: JJ solution -> NL with x0

Gain with JJ: 8.30 dB
Gain with NL: 9.00 dB

Debug output:
1: Solving with Josephson junctions...
  JJ solution norm: 94.79558261550775

2: Solving with inductances...
  L solution norm: 95.81353254610399

3: Solving with NL using L solution as x0...
  NL solution norm: 94.75825194771247



In [11]:
print("\nFlux-pumped JTWPA test comparing JJ and NL")

if set_fork:
    julia_setup.force_revise_update(jl)
    jl.eval("JosephsonCircuits.clear_debug_log()")

result = jl.eval(r"""
    using JosephsonCircuits
    using LinearAlgebra
    
    io = IOBuffer()
    
    function test_flux_jtwpa()
        # Constants
        magnetic_flux_quantum = 2.0678338484619295e-15
        reduced_magnetic_flux_quantum = magnetic_flux_quantum / (2*pi)
        
        # Design parameters from paper
        cutoff_frequency = 46e9  # Hz
        transmission_line_impedance = 50.0
        
        # Calculate component values
        capacitance = 1 / (2 * pi * cutoff_frequency * transmission_line_impedance)
        junction_inductance = transmission_line_impedance / (2 * pi * cutoff_frequency)
        critical_current = reduced_magnetic_flux_quantum / junction_inductance
        
        # Junction capacitance
        critical_current_density = 3e6  # A/m^2
        jj_area = critical_current / critical_current_density
        jj_cap_density = 50e-15 / (1e-6)^2  # F/m^2
        jj_capacitance = jj_cap_density * jj_area
        
        # Circuit parameters
        nr_cells = 500
        modulation_parameter = 0.06
        
        # Coupling and inductances
        coupling = 0.02  # M/L'
        mutual_inductance = coupling * junction_inductance
        linear_squid_loop_inductance = mutual_inductance^2 / junction_inductance
        
        # DC bias and pump
        optimal_dc_flux = magnetic_flux_quantum / 3
        Idc = optimal_dc_flux / mutual_inductance * 0.95
        Ip = modulation_parameter * Idc
        
        pump_line_inductance = 1.1 * junction_inductance
        
        # Frequencies
        pump_frequency = 20e9  # Hz
        signal_frequency = 10e9  # Half pump frequency
        
        # Test parameters
        ws = 2*pi*signal_frequency
        wp = (2*pi*pump_frequency,)
        sources = [
            (mode=(0,), port=3, current=Idc),  # DC bias on pump port
            (mode=(1,), port=3, current=Ip)    # Pump on pump port
        ]
        Npumpharmonics = (8,)
        Nmodulationharmonics = (4,)
        
        println(io, "=== Flux-pumped JTWPA Test ===")
        println(io, "Number of cells: $nr_cells")
        println(io, "Pump: $(pump_frequency/1e9) GHz @ $(Ip*1e6) μA")
        println(io, "DC bias: $(Idc*1e6) μA")
        println(io, "Signal: $(signal_frequency/1e9) GHz")
        
        ########################## JJ Circuit ############################
        
        circuit_jj = Tuple{String,String,String,Any}[]
        
        # Build circuit with JJs
        node = 1
        
        # P1: Start of transmission line
        push!(circuit_jj, ("P$(node)_0", "$node", "0", 1))
        push!(circuit_jj, ("R$(node)_0", "$node", "0", 50.0))
        
        # P3: Start of pump line
        node_pump = node + 1
        push!(circuit_jj, ("P$(node_pump)_0", "$node_pump", "0", 3))
        push!(circuit_jj, ("R$(node_pump)_0", "$node_pump", "0", 50.0))
        
        # Build cells
        for cell in 1:nr_cells
            # Transmission line capacitor
            if cell == 1
                push!(circuit_jj, ("C$(node)_0", "$node", "0", capacitance/2))
            else
                push!(circuit_jj, ("C$(node)_0", "$node", "0", capacitance))
            end
            
            # DC-SQUID: two JJs with small inductance in loop
            push!(circuit_jj, ("Lj_a$(node)_$(node+3)", "$node", "$(node+3)", junction_inductance))
            push!(circuit_jj, ("Cj_a$(node)_$(node+3)", "$node", "$(node+3)", jj_capacitance))
            push!(circuit_jj, ("L$(node)_$(node+2)", "$node", "$(node+2)", linear_squid_loop_inductance))
            push!(circuit_jj, ("Lj_b$(node+2)_$(node+3)", "$(node+2)", "$(node+3)", junction_inductance))
            push!(circuit_jj, ("Cj_b$(node+2)_$(node+3)", "$(node+2)", "$(node+3)", jj_capacitance))
            
            # Pump line
            push!(circuit_jj, ("L$(node_pump)_$(node_pump+3)", "$node_pump", "$(node_pump+3)", pump_line_inductance))
            if cell == 1
                push!(circuit_jj, ("C$(node_pump)_0", "$node_pump", "0", pump_line_inductance/transmission_line_impedance^2/2))
            else
                push!(circuit_jj, ("C$(node_pump)_0", "$node_pump", "0", pump_line_inductance/transmission_line_impedance^2))
            end
            
            # Mutual coupling
            push!(circuit_jj, ("K$(node)", "L$(node)_$(node+2)", "L$(node_pump)_$(node_pump+3)", 0.999))
            
            node += 3
            node_pump += 3
        end
        
        # End capacitors and ports
        push!(circuit_jj, ("C$(node)_0", "$node", "0", capacitance/2))
        push!(circuit_jj, ("P$(node)_0", "$node", "0", 2))  # P2: End of transmission line
        push!(circuit_jj, ("R$(node)_0", "$node", "0", 50.0))
        
        push!(circuit_jj, ("C$(node_pump)_0", "$node_pump", "0", pump_line_inductance/transmission_line_impedance^2/2))
        push!(circuit_jj, ("P$(node_pump)_0", "$node_pump", "0", 4))  # P4: End of pump line
        push!(circuit_jj, ("R$(node_pump)_0", "$node_pump", "0", 50.0))
        push!(circuit_jj, ("L$(node_pump)_0", "$node_pump", "0", 20.0e-9))  # Ground inductance
        
        # Empty circuitdefs since all values are numeric
        circuitdefs = Dict{Symbol,Complex{Float64}}()
        
        println(io, "\n1: Solving with Josephson junctions...")
        hb_jj = hbsolve(
            ws, wp, sources, Nmodulationharmonics,
            Npumpharmonics, circuit_jj, circuitdefs;
            dc=true, threewavemixing=true, fourwavemixing=true,
            switchofflinesearchtol=0.0, alphamin=1e-7, iterations=200
        )
        
        jj_nodeflux = vec(hb_jj.nonlinear.nodeflux)
        println(io, "  JJ solution norm: $(norm(jj_nodeflux))")
        
        ########################## NL Circuit ############################
        
        circuit_nl = Tuple{String,String,String,Any}[]
        
        # Build same circuit with NL elements
        node = 1
        
        # P1: Start of transmission line
        push!(circuit_nl, ("P$(node)_0", "$node", "0", 1))
        push!(circuit_nl, ("R$(node)_0", "$node", "0", 50.0))
        
        # P3: Start of pump line  
        node_pump = node + 1
        push!(circuit_nl, ("P$(node_pump)_0", "$node_pump", "0", 3))
        push!(circuit_nl, ("R$(node_pump)_0", "$node_pump", "0", 50.0))
        
        # Build cells with NL elements
        for cell in 1:nr_cells
            # Transmission line capacitor
            if cell == 1
                push!(circuit_nl, ("C$(node)_0", "$node", "0", capacitance/2))
            else
                push!(circuit_nl, ("C$(node)_0", "$node", "0", capacitance))
            end
            
            # DC-SQUID with NL elements
            push!(circuit_nl, ("NL_a$(node)_$(node+3)", "$node", "$(node+3)", 
                "poly $(junction_inductance), 0.0, 0.5"))
            push!(circuit_nl, ("Cj_a$(node)_$(node+3)", "$node", "$(node+3)", jj_capacitance))
            push!(circuit_nl, ("L$(node)_$(node+2)", "$node", "$(node+2)", linear_squid_loop_inductance))
            push!(circuit_nl, ("NL_b$(node+2)_$(node+3)", "$(node+2)", "$(node+3)", 
                "poly $(junction_inductance), 0.0, 0.5"))
            push!(circuit_nl, ("Cj_b$(node+2)_$(node+3)", "$(node+2)", "$(node+3)", jj_capacitance))
            
            # Pump line (same as JJ)
            push!(circuit_nl, ("L$(node_pump)_$(node_pump+3)", "$node_pump", "$(node_pump+3)", pump_line_inductance))
            if cell == 1
                push!(circuit_nl, ("C$(node_pump)_0", "$node_pump", "0", pump_line_inductance/transmission_line_impedance^2/2))
            else
                push!(circuit_nl, ("C$(node_pump)_0", "$node_pump", "0", pump_line_inductance/transmission_line_impedance^2))
            end
            
            # Mutual coupling
            push!(circuit_nl, ("K$(node)", "L$(node)_$(node+2)", "L$(node_pump)_$(node_pump+3)", 0.999))
            
            node += 3
            node_pump += 3
        end
        
        # End capacitors and ports
        push!(circuit_nl, ("C$(node)_0", "$node", "0", capacitance/2))
        push!(circuit_nl, ("P$(node)_0", "$node", "0", 2))
        push!(circuit_nl, ("R$(node)_0", "$node", "0", 50.0))
        
        push!(circuit_nl, ("C$(node_pump)_0", "$node_pump", "0", pump_line_inductance/transmission_line_impedance^2/2))
        push!(circuit_nl, ("P$(node_pump)_0", "$node_pump", "0", 4))
        push!(circuit_nl, ("R$(node_pump)_0", "$node_pump", "0", 50.0))
        push!(circuit_nl, ("L$(node_pump)_0", "$node_pump", "0", 20.0e-9))
        
        println(io, "\n2: Solving with NL elements...")
        hb_nl = hbsolve(
            ws, wp, sources, Nmodulationharmonics,
            Npumpharmonics, circuit_nl, circuitdefs;
            dc=true, threewavemixing=true, fourwavemixing=true,
            switchofflinesearchtol=0.0, alphamin=1e-7, iterations=200
        )
        
        nl_nodeflux = vec(hb_nl.nonlinear.nodeflux)
        println(io, "  NL solution norm: $(norm(nl_nodeflux))")
        
        # Calculate gains
        S21_jj = hb_jj.linearized.S(
            outputmode=(0,), outputport=2,
            inputmode=(0,), inputport=1, freqindex=1
        )
        S21_nl = hb_nl.linearized.S(
            outputmode=(0,), outputport=2,
            inputmode=(0,), inputport=1, freqindex=1
        )
        
        gain_jj = 10*log10(abs2(S21_jj))
        gain_nl = 10*log10(abs2(S21_nl))
        
        return (gain_jj, gain_nl, String(take!(io)))
    end
    
    test_flux_jtwpa()
""")

gain_jj, gain_nl, debug_output = result

print("="*60)
print("\nFLUX-PUMPED JTWPA TEST")
print("="*60)
print(f"\nGain with JJ: {gain_jj:.2f} dB")
print(f"Gain with NL: {gain_nl:.2f} dB")
print(f"Gain difference: {gain_nl - gain_jj:.2f} dB")

print("\nDebug output:")
print(debug_output)

if set_fork:
    debug_msgs = jl.eval("JosephsonCircuits.get_debug_log()")
    for msg in debug_msgs:
        print(f"  {msg}")


Flux-pumped JTWPA test comparing JJ and NL
✅ Forced Revise update complete - changes should be reflected now
✅ Also triggered precompilation of JosephsonCircuits fork

FLUX-PUMPED JTWPA TEST

Gain with JJ: -0.05 dB
Gain with NL: 20.79 dB
Gain difference: 20.84 dB

Debug output:
=== Flux-pumped JTWPA Test ===
Number of cells: 500
Pump: 20.0 GHz @ 11.355517764297193 μA
DC bias: 189.2586294049532 μA
Signal: 10.0 GHz

1: Solving with Josephson junctions...
  JJ solution norm: 937862.9544741699

2: Solving with NL elements...
  NL solution norm: 939070.2759024018

